# create a small chatbot

In [7]:
import numpy as np
import pandas as pd
import tensorflow.keras as tk
import tensorflow.compat.v1 as tf
import tensorflow as tens
import nltk
import speech_recognition as sr
from gtts import gTTS
import os
import datetime as dt

In [42]:
import time

In [72]:
import transformers
npl = transformers.pipeline("conversational",model="microsoft/DialoGPT-medium")

In [85]:
class Chatbot:
    def __init__(self,name,lang = 'en'):
        self.name = name
        print("---- \tstarting {}\t ----".format(self.name))
        self.pad_token_id = 50256
        self.error_token = False
        self.chat_responses()
        self.lang = lang
    def chat_responses(self):
        self.greetings_only = [
            "hmm...","yes?","good {}!, how can I help you?".format(self.time_category())
        ]
        self.greetings = [
            "Hello","good {}!".format(self.time_category()),"yes,"
        ]
        self.appreciation = [
            "Your Welcome!","AnyTime","No Problem!","Cool!","I am here if you need me"
        ]
        self.error_response = [
            'Sorry, come again?','Sorry I didnt get you, can you try again?',"I didnt understand what you were saying"
        ]
        self.parting = [
            'It was great talking to you!','Lets talk again later!','Bye Bye'
        ]
        self.only_greet = [
            "hello {}".format(self.name),"{}".format(self.name),"hey {}".format(self.name)
        ]
    def wakeup(self,text):
        return True if self.name in text.lower() else False
    def speech_to_text(self):
        recogniser = sr.Recognizer()
        with sr.Microphone() as mic:
            recogniser.adjust_for_ambient_noise(mic,duration=1)
            print("Listening")
            audio = recogniser.listen(mic)
            self.error_token = True
        try:
            self.error_token = False
            self.text = recogniser.recognize_google(audio)
            print("you said: {}".format(self.text))
            return self.text
        except:
            raise ConnectionError("error Found")
    def text_to_speech(self,text):
        print("{:1} --> {:2}".format(self.name,text))
        speaker = gTTS(text=text,lang=self.lang,slow=False)
        speaker.save('res.mp3')
        os.system('start res.mp3')
        statbuff = os.stat('res.mp3')
        sz = statbuff.st_size / 1024
        duration = sz / 200
        time.sleep(int(60*duration))
        os.remove('res.mp3')
    def actiontime(self):
        return "It is now " + dt.datetime.now().strftime("%Y/%m/%d %H:%M")
    @staticmethod
    def __choice(arr):
        return np.random.choice(arr)
    @staticmethod
    def time_category():
        tag = ""
        k = int(dt.datetime.now().time().strftime("%H"))
        if 0 <= k <= 3 or 17 <= k <= 23:
            tag = "Evening"
        elif 4 <= k <= 11:
            tag = "Morning"
        elif 12 <= k <= 16:
            tag = "Afternoon"

        return tag
    def language_model(self):
        output = str(npl(transformers.Conversation(self.text),pad_token_id=self.pad_token_id))
        return output[output.find('bot >>')+6:].strip()
    def start(self):
        game = True
        while game:
            self.speech_to_text()
            res = ''
            somethingelse = True
            complexbye= any(i in self.text.lower() for i in ['bye', 'see you later', 'talk to you later'])
            if self.wakeup(self.text):
                if self.text in self.only_greet:
                    res = self.__choice(self.greetings_only)
                else:
                    res = res + self.__choice(self.greetings)
                somethingelse = False
            if 'time' in self.text.lower():
                res = res +" "+ self.actiontime() +"."
                somethingelse = False
            if any(i in self.text.lower() for i in ["thank you","thanks","thank"]):
                res = res + " " + self.__choice(self.appreciation)
                somethingelse = False
            if self.text.lower() in ['exit','close','bye','bye bye','see you later', 'talk to you later'] or complexbye:
                res = self.__choice(self.parting)
                somethingelse = False
                game = False
            if somethingelse:
                if self.error_token:
                    res = self.__choice(self.error_response)
                else:
                    res = self.language_model()
            self.text_to_speech(res)
        print("---- \tClosing down {}\t ----".format(self.name))

In [86]:
cl = Chatbot(name="crayons")
cl.start()

---- 	starting crayons ----
Listening


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


you said: hello
crayons --> Hello! :D
Listening


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


you said: what is your name
crayons --> I'm not sure, but I think it's a reference to the movie The Big Lebowski.
Listening
you said: ok bye
crayons --> Lets talk again later!
---- 	Closing down crayons ----


In [37]:
npl(transformers.Conversation("tell me 2+1"),pad_token_id=50256)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: 8250860b-54d0-42cd-9a50-643ebee8a27a 
user >> tell me 2+1 
bot >> I'm not sure what that means, but I'm sure it's a reference to something. 

Listening
you said: hello


In [12]:
k = "he".join("f")
k

'f'